In [7]:
import tensorflow as tf
import odl


In [11]:

# Define the ODL operator
space = odl.uniform_discr([-64, -64], [64, 64], [128, 128],
                        dtype='float32')
geometry = odl.tomo.parallel_beam_geometry(space, num_angles=30)
ray_transform = odl.tomo.RayTransform(space, geometry)


In [16]:

# Define the forward pass function
def forward_pass(x):
    x_odl = odl.phantom.shepp_logan(x, True)  # Convert TensorFlow tensor to ODL array
    y_odl = ray_transform(x_odl)  # Apply ODL operator
    y_tf = tf.convert_to_tensor(y_odl.asarray())  # Convert ODL array to TensorFlow tensor
    return y_tf


In [17]:

# Create a custom TensorFlow layer
class ODLForwardLayer(tf.keras.layers.Layer):
    def __init__(self, ray_transform):
        super(ODLForwardLayer, self).__init__()
        self.ray_transform = ray_transform

    def call(self, inputs):
        return forward_pass(inputs)


In [18]:

# Use the custom layer in a model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(128, 128, 1)),
    ODLForwardLayer(ray_transform),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128 * 128, activation='linear')
])

AttributeError: Exception encountered when calling layer "odl_forward_layer_2" (type ODLForwardLayer).

in user code:

    File "/var/folders/c1/jljg7x_d7fb61svm70dxncg40000gn/T/ipykernel_18543/297015694.py", line 8, in call  *
        return forward_pass(inputs)
    File "/var/folders/c1/jljg7x_d7fb61svm70dxncg40000gn/T/ipykernel_18543/4006368727.py", line 3, in forward_pass  *
        x_odl = odl.phantom.shepp_logan(x, True)  # Convert TensorFlow tensor to ODL array
    File "/Users/saityada/miniconda3/envs/illpropinverse/lib/python3.11/site-packages/odl/phantom/transmission.py", line 153, in shepp_logan  *
        ellipsoids = shepp_logan_ellipsoids(space.ndim, modified)

    AttributeError: 'Tensor' object has no attribute 'ndim'


Call arguments received by layer "odl_forward_layer_2" (type ODLForwardLayer):
  • inputs=tf.Tensor(shape=(None, 128, 128, 1), dtype=float32)